In [1]:
import json
import pandas as pd
import numpy as np
import os
import seaborn as sns
import re
from collections import *
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.sparse import csr_matrix
import time
import random
import webbrowser
import pickle, gzip, joblib, shelve
import tkinter as tk
from tkinter import ttk, font
import threading, time

In [2]:
directory_path = 'data/raw'
filenames = sorted(os.listdir(directory_path))
print(f"{len(filenames) * 1000} playlists")

1000000 playlists


In [3]:
# looking at only the first 30,000 playlists
fullpaths = [directory_path + '/' + f for f in filenames][0:200]

In [4]:
min_tracks_per_playlist = 5
max_tracks_per_playlist = 150
min_albums_per_playlist = 2
min_artists_per_playlist = 2

In [ ]:
# song_relationships = {}

# t = 0
# iteration_times = []

# for idx, path in enumerate(fullpaths):
#     start_time = time.time()
#     if idx % 5 == 0 and idx > 0:
#         print(f"processed {idx-5}-{idx} - time taken {t:.2f}")
#         iteration_times.append(t)
#         t = 0
#     with open(path) as f:
#         mpd_slice = json.load(f)
#     playlists_data = mpd_slice['playlists']

#     for idx, playlist in enumerate(playlists_data):
#         # r = playlist['num_albums'] / playlist['num_tracks']
#         # if r < 0.0:
#         #     continue
#         songs = set(track['track_uri'] for track in playlist['tracks'])
#         t_per_p = len(songs)
#         albums = set(track['album_uri'] for track in playlist['tracks'])
#         alb_per_p = len(albums)
#         artists = set(track['artist_uri'] for track in playlist['tracks'])
#         art_per_p = len(artists)

#         if (min_tracks_per_playlist >= t_per_p) or \
#              (t_per_p >= max_tracks_per_playlist) or \
#                 (min_albums_per_playlist >= alb_per_p) or \
#                     (min_artists_per_playlist >= art_per_p):
#             continue

#         for song in songs:
#             if song not in song_relationships:
#                 song_relationships[song] = {}
                
#             for related_song in songs:
#                 if related_song != song:  # avoid self-relationship
#                     if related_song in song_relationships[song]:
#                         song_relationships[song][related_song] += 1
#                     else:
#                         song_relationships[song][related_song] = 1
#     end_time = time.time()
#     t += end_time - start_time

# print(f'Number of songs processed: {len(song_relationships)}')


In [5]:
song_relationships = {}

t = 0
iteration_times = []

for idx, path in enumerate(fullpaths):
    start_time = time.time()
    if idx % 10 == 0 and idx > 0:
        print(f"processed {idx-10}-{idx} - time taken {t:.2f}")
        iteration_times.append(t)
        t = 0
    with open(path) as f:
        mpd_slice = json.load(f)
    playlists_data = mpd_slice['playlists']

    for idx, playlist in enumerate(playlists_data):
        # r = playlist['num_albums'] / playlist['num_tracks']
        # if r < 0.0:
        #     continue
        songs = set(track['track_uri'] for track in playlist['tracks'])
        t_per_p = len(songs)
        albums = set(track['album_uri'] for track in playlist['tracks'])
        alb_per_p = len(albums)
        artists = set(track['artist_uri'] for track in playlist['tracks'])
        art_per_p = len(artists)

        if (min_tracks_per_playlist >= t_per_p) or \
             (t_per_p >= max_tracks_per_playlist) or \
                (min_albums_per_playlist >= alb_per_p) or \
                    (min_artists_per_playlist >= art_per_p):
            continue
        
        temp_relationships = {}

        for song in songs:
            if song not in temp_relationships:
                temp_relationships[song] = {}
            
            for related_song in songs:
                if related_song != song:  # avoid self-relationship
                    if related_song in temp_relationships[song]:
                        temp_relationships[song][related_song] += 1
                    else:
                        temp_relationships[song][related_song] = 1

        for song, relations in temp_relationships.items():
            if song not in song_relationships:
                song_relationships[song] = dict(relations)
            else:
                for related_song, count in relations.items():
                    if related_song in song_relationships[song]:
                        song_relationships[song][related_song] += count
                    else:
                        song_relationships[song][related_song] = count

    end_time = time.time()
    t += end_time - start_time

song_relationships = {key: list(value.items()) for key, value in tqdm(song_relationships.items())}
    
print(f'{len(song_relationships)} songs processed')

# save_path = os.path.join('song_data', 'song_relationships.gz')

# with gzip.open(save_path, 'wb') as f:
#     pickle.dump(song_relationships, f, protocol = pickle.HIGHEST_PROTOCOL)
# print(f"'song_relationships' saved to {save_path}")

processed 0-10 - time taken 17.60
processed 10-20 - time taken 21.18
processed 20-30 - time taken 21.06
processed 30-40 - time taken 21.40
processed 40-50 - time taken 22.20
processed 50-60 - time taken 25.65
processed 60-70 - time taken 23.48
processed 70-80 - time taken 24.03
processed 80-90 - time taken 24.21
processed 90-100 - time taken 25.43
processed 100-110 - time taken 27.53
processed 110-120 - time taken 29.41
processed 120-130 - time taken 34.93
processed 130-140 - time taken 40.51
processed 140-150 - time taken 42.18
processed 150-160 - time taken 46.70
processed 160-170 - time taken 51.54
processed 170-180 - time taken 54.33
processed 180-190 - time taken 55.23


100%|██████████| 821079/821079 [06:31<00:00, 2098.76it/s] 


821079 songs processed
'song_relationships' saved to song_data/song_relationships.gz


In [ ]:
# to load song_relationships from storage

save_path = os.path.join('song_data', 'song_relationships.gz')

with gzip.open(save_path, 'rb') as f:
    song_relationships = pickle.load(f)

In [6]:
song_data_map = {}
for idx, path in enumerate(fullpaths):
    if idx % 10 == 0 and idx > 0:
        print(f"Processed {idx-10}-{idx}")
    with open(path) as f:
        mpd_slice = json.load(f)
    playlists_data = mpd_slice['playlists']
    for playlist in playlists_data:
        for track in playlist['tracks']:
            song_uri = track['track_uri']
            song_name = track['track_name']
            album_name = track['album_name']
            artist_name = track['artist_name']
            if song_uri in song_relationships:
                song_data_map[song_uri] = {'song_name': song_name, 'album_name': album_name, 'artist_name': artist_name}

save_path = os.path.join('song_data', 'song_data_map.gz')

with gzip.open(save_path, 'wb') as f:
    pickle.dump(song_data_map, f)
print(f"'song_data_map' saved to {save_path}")

Processed 0-10
Processed 10-20
Processed 20-30
Processed 30-40
Processed 40-50
Processed 50-60
Processed 60-70
Processed 70-80
Processed 80-90
Processed 90-100
Processed 100-110
Processed 110-120
Processed 120-130
Processed 130-140
Processed 140-150
Processed 150-160
Processed 160-170
Processed 170-180
Processed 180-190
'song_data_map' saved to song_data/song_data_map.pkl


In [ ]:
# to load song_data_map from storage

save_path = os.path.join('song_data', 'song_data_map.gz')

with gzip.open(save_path, 'rb') as f:
    song_data_map = pickle.load(f)

In [7]:
max_connections = 0
song_with_most_connections = None

for song, connections in song_relationships.items():
    num_connections = len(connections)
    if num_connections > max_connections:
        max_connections = num_connections
        song_with_most_connections = song

print("Song with the most connections:", song_data_map[song_with_most_connections])
print("Number of connections:", max_connections)


Song with the most connections: {'song_name': 'Closer', 'album_name': 'Closer', 'artist_name': 'The Chainsmokers'}
Number of connections: 60039


In [ ]:
song_indices = {song_uri: idx for idx, song_uri in enumerate(song_relationships.keys())}
num_songs = len(song_relationships)

In [8]:
row_indices = []
col_indices = []
data_sum = []
data = []

for idx, (song_uri, relationships) in enumerate(song_relationships.items()):
    row_idx = song_indices[song_uri]
    # sum_connections = np.sum([x[1] for x in relationships])
    for related_song_uri, count in relationships:
        col_idx = song_indices[related_song_uri]
        row_indices.append(row_idx)
        col_indices.append(col_idx)
        # data_sum.append(count / sum_connections)
        data.append(count)

    if idx % 30000 == 0 and idx > 0:
        print(f"Processed {idx} rows.....")

# transition_matrix = csr_matrix((data_sum, (row_indices, col_indices)), shape=(num_songs, num_songs), dtype=np.float64)
cooccurrence_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(num_songs, num_songs), dtype=np.float64)

save_path = os.path.join('song_data', 'cooccurrence_matrix.gz')
with gzip.open(save_path, 'wb') as f:
    pickle.dump(cooccurrence_matrix, f)
print(f"'cooccurrence_matrix' saved to {save_path}")


Processed 30000 rows.....
Processed 60000 rows.....
Processed 90000 rows.....
Processed 120000 rows.....
Processed 150000 rows.....
Processed 180000 rows.....
Processed 210000 rows.....
Processed 240000 rows.....
Processed 270000 rows.....
Processed 300000 rows.....
Processed 330000 rows.....
Processed 360000 rows.....
Processed 390000 rows.....
Processed 420000 rows.....
Processed 450000 rows.....
Processed 480000 rows.....
Processed 510000 rows.....
Processed 540000 rows.....
Processed 570000 rows.....
Processed 600000 rows.....
Processed 630000 rows.....
Processed 660000 rows.....
Processed 690000 rows.....
Processed 720000 rows.....
Processed 750000 rows.....
Processed 780000 rows.....
Processed 810000 rows.....
'cooccurrence_matrix' saved to song_data/cooccurrence_matrix.pkl


In [ ]:
# to load cooccurrence_matrix from storage

save_path = os.path.join('song_data', 'cooccurrence_matrix.gz')

with gzip.open(save_path, 'rb') as f:
    cooccurrence_matrix = pickle.load(f)

In [ ]:
n = 10

popularity = np.sum(transition_matrix, axis=0)
top_n_indices = np.argsort(popularity.A1)[-n:]

init = np.random.rand(1, num_songs)
init = init / np.sum(init)
probs = [init]
p = csr_matrix(init)

damping = True
if damping:
    damping_factor = 0.85
    random_jump_vector = csr_matrix(np.ones(num_songs)/num_songs)
    for i in tqdm(range(30)):
        p = damping_factor * np.dot(p, transition_matrix) + (1 - damping_factor) * random_jump_vector
        probs.append(p)
else:
    for i in tqdm(range(30)):
        p = np.dot(p, transition_matrix)
        probs.append(p)


plot_data = []
for i in top_n_indices:
    song_name = song_data_map[list(song_relationships)[i]]["song_name"]
    for step_num, step in enumerate(probs):
        plot_data.append({'Iteration': step_num, 'Probability': step[0, i], 'Song': song_name})

df = pd.DataFrame(plot_data)
fig = px.line(df, x='Iteration', y='Probability', color='Song', title='Convergence of Most Popular Songs')
fig.show()

In [ ]:
# A = transition_matrix.copy()
# def user_playlist_vector(playlist_songs, song_indices, num_songs):
#     user_vector = np.zeros(num_songs)
#     for song in playlist_songs:
#         user_vector[song_indices[song]] = 1/len(playlist_songs)
#     return csr_matrix(user_vector)

# def get_recommendation_vector(user_vector, transition_matrix, steps, damping = True, damping_factor = 0.0):
#     p = user_vector
#     if damping:
#         random_jump_vector = csr_matrix(np.ones(num_songs) / num_songs)
#         for _ in range(steps):
#             p = damping_factor * np.dot(p, transition_matrix) + (1 - damping_factor) * user_vector
#     else:
#         for _ in range(steps):
#             p = np.dot(p, transition_matrix)
#     return p.toarray()[0]

# def top_n_recommendations(recommendation_vector, song_data_map, n):
#     top_indices = np.argsort(recommendation_vector)[-n:]
#     top_songs = [song_data_map[list(song_relationships)[i]] for i in top_indices]
#     return top_songs

# def recommend_songs(user_playlist, song_indices, transition_matrix, song_data_map, steps, n=10):
#     user_vector = user_playlist_vector(user_playlist, song_indices, len(song_indices))
#     recommendation_vector = get_recommendation_vector(user_vector, transition_matrix, steps)
#     return top_n_recommendations(recommendation_vector, song_data_map, n)

# top_n_songs = [list(song_relationships)[i] for i in top_n_indices]
# playlist = random.sample(list(song_relationships), 3) #+ top_n_songs

# for i, j in zip(playlist, [song_data_map[x] for x in playlist]):
#     print(i, j)

# recommended_songs = recommend_songs(playlist, song_indices, A, song_data_map, 10, n=5)
# for song in recommended_songs:
#     print(f"Song: {song['song_name']}\nAlbum: {song['album_name']}\nArtist: {song['artist_name']}\n{'-'*40}")    

In [9]:
total_occurrences = np.sum(cooccurrence_matrix)
p_i = np.sum(cooccurrence_matrix, axis=1) / total_occurrences
p_i = np.asarray(p_i).flatten()  # Convert matrix to 1D array
p_ij = cooccurrence_matrix / total_occurrences

pmi_data = np.array(p_ij.data)
k = 1

for i in tqdm(range(len(p_ij.indptr) - 1)):
    for data_idx in range(p_ij.indptr[i], p_ij.indptr[i + 1]):
        j = p_ij.indices[data_idx]
        if pmi_data[data_idx] > 0:  # Avoid log(0)
            original_pmi = np.log2(pmi_data[data_idx] / (p_i[i] * p_i[j]))
            pmi_data[data_idx] = original_pmi - (-(k - 1) * np.log2(pmi_data[data_idx]))
        else:
            pmi_data[data_idx] = 0  # Or any other default value for log(0)

pmi_matrix = csr_matrix((pmi_data, p_ij.indices, p_ij.indptr), shape=cooccurrence_matrix.shape)

save_path = os.path.join('song_data', 'pmi_matrix.gz')
with gzip.open(save_path, 'wb') as f:
    pickle.dump(pmi_matrix, f)
print(f"'pmi_matrix' saved to {save_path}")

100%|██████████| 821079/821079 [16:23<00:00, 834.48it/s] 


'pmi_matrix' saved to song_data/pmi_matrix.gz


In [ ]:
# to load pmi_matrix from storage

save_path = os.path.join('song_data', 'pmi_matrix.gz')

with gzip.open(save_path, 'rb') as f:
    pmi_matrix = pickle.load(f)

In [ ]:
# pmi_data = np.array(p_ij.data)

# for i in tqdm(range(len(p_ij.indptr) - 1)):
#     for data_idx in range(p_ij.indptr[i], p_ij.indptr[i + 1]):
#         j = p_ij.indices[data_idx]
#         if pmi_data[data_idx] > 0:  # Avoid log(0)
#             pmi_data[data_idx] = np.log2(pmi_data[data_idx] / (p_i[i] * p_i[j]))
#         else:
#             pmi_data[data_idx] = 0  # Or any other default value for log(0)

# pmi_matrix = csr_matrix((pmi_data, p_ij.indices, p_ij.indptr), shape=cooccurence_matrix.shape)

In [10]:
def user_playlist_vector(playlist_songs, song_indices, num_songs):
    user_vector = np.zeros(num_songs)
    for song in playlist_songs:
        user_vector[song_indices[song]] = 1
    return csr_matrix(user_vector)

def compute_scores(user_vector, pmi_matrix):
    scores = user_vector.dot(pmi_matrix)
    return scores.toarray()[0]

def get_top_recommendations(scores, song_data_map, n=10):
    top_indices = np.argsort(scores)[-n:]
    top_songs = [song_data_map[list(song_relationships)[i]] for i in top_indices]
    return top_songs

def recommend_songs_pmi(user_playlist, song_indices, pmi_matrix, song_data_map, n=10):
    user_vector = user_playlist_vector(user_playlist, song_indices, num_songs)
    user_vector_csr = csr_matrix(user_vector)
    scores = compute_scores(user_vector_csr, pmi_matrix)
    return get_top_recommendations(scores, song_data_map, n)

In [15]:
class SongRecommendationApp(tk.Tk):
    def __init__(self, song_data_map):
        super().__init__()

        default_font = font.nametofont("TkDefaultFont")
        default_font.configure(family="Courier")

        self.song_data_map = song_data_map
        self.uri_map = {self.format_song_display(song_info): uri for uri, song_info in song_data_map.items()}
        self.playlist_data = []  # Store song data for sorting

        # Filter Frame
        self.filter_frame = ttk.Frame(self)
        self.filter_frame.pack(pady=10)

        # Label and Entry for Song
        self.song_label = ttk.Label(self.filter_frame, text="Song")
        self.song_label.grid(row=0, column=0, padx=5)
        self.song_entry = ttk.Entry(self.filter_frame)
        self.song_entry.grid(row=1, column=0, padx=5)

        # Label and Entry for Artist
        self.artist_label = ttk.Label(self.filter_frame, text="Artist")
        self.artist_label.grid(row=0, column=1, padx=5)
        self.artist_entry = ttk.Entry(self.filter_frame)
        self.artist_entry.grid(row=1, column=1, padx=5)

        # Label and Entry for Album
        self.album_label = ttk.Label(self.filter_frame, text="Album")
        self.album_label.grid(row=0, column=2, padx=5)
        self.album_entry = ttk.Entry(self.filter_frame)
        self.album_entry.grid(row=1, column=2, padx=5)

        # Debounce logic
        self.last_time = time.time()

        self.search_button = ttk.Button(self.filter_frame, text="Search", command=self.display_search_results)
        self.search_button.grid(row=2, columnspan=3, pady=10)

        width = 200
        # Songs Listbox
        self.songs_listbox = tk.Listbox(self, selectmode=tk.SINGLE, width=width, font=("Courier", 10))
        self.songs_listbox.pack(pady=10)

        # Drag & Drop functionality
        self.songs_listbox.bind('<<ListboxSelect>>', self.add_to_playlist)

        # Playlist Listbox
        self.playlist_listbox = tk.Listbox(self, bg="lightblue", selectmode=tk.SINGLE, width=width, font=("Courier", 10))
        self.playlist_listbox.pack(pady=10)

        # Number of recommendations
        self.n_label = ttk.Label(self, text="Number of Recommendations:")
        self.n_label.pack(pady=5)
        self.n_entry = ttk.Entry(self)
        self.n_entry.pack(pady=5)

        # Button to generate recommendations
        self.btn_recommend = ttk.Button(self, text="Generate Recommendations", command=self.generate_recommendations)
        self.btn_recommend.pack(pady=10)

        # Recommendations Listbox
        self.recommendations_listbox = tk.Listbox(self, bg="lightgreen", selectmode=tk.SINGLE, width=width, font=("Courier", 10))
        self.recommendations_listbox.pack(pady=10)

        self.recommendations_listbox.bind('<Double-Button-1>', self.open_in_spotify)

        self.btn_refresh = ttk.Button(self, text="Refresh", command=self.refresh)
        self.btn_refresh.pack(pady=10)
    
    def refresh(self):
        # Clear all fields
        self.song_entry.delete(0, tk.END)
        self.artist_entry.delete(0, tk.END)
        self.album_entry.delete(0, tk.END)
        self.n_entry.delete(0, tk.END)
        
        # Clear listboxes
        self.songs_listbox.delete(0, tk.END)
        self.playlist_listbox.delete(0, tk.END)
        self.recommendations_listbox.delete(0, tk.END)

    def open_in_spotify(self, event):
            selected_index = self.recommendations_listbox.curselection()
            if selected_index:
                selected_song = self.recommendations_listbox.get(selected_index)
                song_uri = self.uri_map[selected_song]
                uri = song_uri.split(':')[-1]
                webbrowser.open(f"https://open.spotify.com/track/{uri}")
                
    def display_search_results(self):
        song_query = self.song_entry.get().lower()
        artist_query = self.artist_entry.get().lower()
        album_query = self.album_entry.get().lower()

        self.songs_listbox.delete(0, tk.END)
        results = []  # Store the filtered results first

        for uri, song_info in self.song_data_map.items():
            if song_query in song_info['song_name'].lower() and artist_query in song_info['artist_name'].lower() and album_query in song_info['album_name'].lower():
                display_name = self.format_song_display(song_info)
                results.append(display_name)

        # Sort by album name
        results.sort(key=lambda x: self.song_data_map[self.uri_map[x]]['album_name'])

        # Display the sorted results
        for display_name in results:
            self.songs_listbox.insert(tk.END, display_name)

        if len(results) > 200:  # If you want to limit the displayed results
            self.songs_listbox.delete(201, tk.END)

    def format_song_display(self, song_info):
        formatted_str = "{:<65}{:<35}{:<35}"
        f_string = formatted_str.format(song_info['song_name'], song_info['artist_name'], song_info['album_name'])
        return f_string

    def add_to_playlist(self, event):
        selected_index = self.songs_listbox.curselection()
        if selected_index:  # This checks if there's any selection at all
            selected_song = self.songs_listbox.get(selected_index)
            if selected_song not in self.playlist_listbox.get(0, tk.END):  # Prevent duplicates
                self.playlist_listbox.insert(tk.END, selected_song)

    def generate_recommendations(self):
        playlist_display_names = list(self.playlist_listbox.get(0, tk.END))
        playlist_uris = [self.uri_map[display_name] for display_name in playlist_display_names]  # Extract URIs

        n = int(self.n_entry.get())
        recommended_songs = recommend_songs_pmi(playlist_uris, song_indices, pmi_matrix, song_data_map, n)

        self.recommendations_listbox.delete(0, tk.END)
        for song in recommended_songs:
            formatted_song = self.format_song_display(song)
            self.recommendations_listbox.insert(tk.END, formatted_song)

if __name__ == "__main__":
    app = SongRecommendationApp(song_data_map)
    app.mainloop()


In [ ]:
playlist = random.sample(list(song_relationships), 1)

for i, j in zip(playlist, [song_data_map[x] for x in playlist]):
    print(f"Song: {j['song_name']}\nAlbum: {j['album_name']}\nArtist: {j['artist_name']}\n{'-'*40}")

print(f'\n{"-"*60}')
recommended_songs = recommend_songs_pmi(playlist, song_indices, pmi_matrix, song_data_map, n=8)
for song in recommended_songs:
    print(f"Song: {song['song_name']}\nAlbum: {song['album_name']}\nArtist: {song['artist_name']}\n{'-'*40}")    